# 数据获取和存储

+ 在这一节中，我们主要介绍如何获取和读入数据，以及如何存储数据。
+ 其中网络数据抓取等内容可以参考学习Beautiful soup,tushare,pandas_datareader等包。
+ 程序在3.3或更高版本中，如果在Python 2.7运行，可能会有所差别，特别是几个涉及中文字符的例子。

+ 下面是基本设定，需要导入的包和一些简单设定。

In [1]:
import os
import sys

import numpy as np
from numpy.random import randn
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt


np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

## Reading and Writing Data in Text Format

+ 读写文本格式的数据， 在所有pandas 包中关于读取的函数中个，read_csv 和read_table 是使用最多的。
+ 在读取存储在文本文件中的数据时，首先要关注一下 文档路径 以及 当前 notebook的工作目录

In [2]:
%pwd

'D:\\teaching\\金融数据分析datafin'

+ 打开文件，大致了解一下文档中数据的格式

In [3]:
#运用dos命令
!type data\ex1.csv  

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


+ 逗号分隔的数据，可以用read_csv函数读入

In [4]:
df = pd.read_csv('data/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 也可以用read_table读入，只是这时需要 设置 数据之间的分隔符参数： sep=','

In [5]:
pd.read_table('data/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 看看ex2文件，其中第一行不是变量名

In [6]:
!type data\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


+ 因此需要指定和变量名相关的参数：header=None 或者 names=['a', 'b', 'c', 'd', 'message'] 等

In [7]:
pd.read_csv('data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 我们也可以指定DataFrame 的 行索引关键字：index_col='message'  说明用message这列作为行索引关键字
+ 也可以用index_col=4,message的列号指定其为关键字

In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


+ 再看下面的例子

In [10]:
!type data\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


+ index关键字可以是两列，这样就有多层索引。

In [11]:
parsed = pd.read_csv('data/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

+ 分隔符可以有多种形式，比如如果分隔符为数量不等长空格符，则用正则表达式 '\s+'表示分隔符格式

In [12]:
list(open('data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_csv('data/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


+ pandas自动推断出第一行为列名，第一列为index. 

+ 有时候文档中会有说明文字，需要指定哪些行不用读入。
+ 选项 skiprows给出跳过的行。

In [14]:
!type data\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [15]:
pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ pandas会自动处理常见的丢失值，不过也可以指定哪些值是缺失值
+ 我们甚至可以给不同列指定不同的缺失值。
+ 下面的例子中有标注了的缺失值 NA；也有没有标注的，连续两个逗号表示中间缺了一个数

In [16]:
!type data\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [17]:
result = pd.read_csv('data/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 指定缺失值的实例

In [18]:
result = pd.read_csv('data/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


+ 处理带中文的表格，Python 3 更容易

In [20]:
Astock=pd.read_table("data/A.txt",encoding="gbk")
Astock.columns 

Index(['代码', '名称', '涨幅%%', '涨跌', '最高%%', '现价', '开盘%%', '现量', '投资收益', '市现率',
       '市净率', '市盈(动)', '每股未分配', '每股未分配.1', '昨收', '资产负债率%%', '人均持股', '净利润率%%',
       '细分行业', '流通股本(万)', '地区', '振幅%%', 'Unnamed: 22'],
      dtype='object')

### 逐块读入数据

当文件很大的时候，比如上百M。可能逐块读入更好。下面的例子有10000行数据

In [21]:
result=pd.read_csv('data/ex6.csv')
pd.read_csv('data/ex6.csv', nrows=5) #只读前5行

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 设置 chunksize实现逐块阅读
+ 我们可以通过.read　方法将前面若干行读出来。

In [22]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)
chunker.read(5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 也可以用循环分块处理
+ 下面例子统计key列中各字母出现的频数，这里用到了Series求和的对齐功能。
+ .value_counts()是统计频数的方法

In [23]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)
tot = Series([],dtype=pd.Int16Dtype())
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:5]

E    368
X    364
L    346
O    343
Q    340
dtype: Int64

### 将数据写出到文本格式

首先看看前面处理过的csv文件

In [24]:
data = pd.read_csv('data/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 用数据data的方法to_csv写出到文件

In [25]:
data.to_csv('data/out.csv')
!type data\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


+ sys.stdout 是输出到屏幕，看先观察一下效果
+ sep='|' 分割符为 | 

In [26]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


+ na_rep='NULL' 表示用"NULL"代替 缺失值位置

In [27]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


+  index=False, header=False 说明不输出行和列索引

In [28]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


+ 当然也可以重新指定列名和索引

In [29]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


+ 下面是一个时间序列索引数据读写，我们会在后面专门介绍

In [30]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('data/tseries.csv')
!type data\tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [31]:
pd.read_csv('data/tseries.csv', parse_dates=True,names=["date","num"],index_col=["date"],skiprows=[0])

,num
date,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 二进制数据 格式  Binary data formats

+ Python 数据的二进制存储文档一般是带有 .pkl 扩展名的文件。 
+ 在pandas中，有读入数据的方法，而数据对象对象DataFrame和Series也有将数据写入二进制文件的方法。
+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [32]:
frame = pd.read_csv('data/ex1.csv')
frame
frame.to_pickle('data/frame_pickle')

In [33]:
pd.read_pickle('data/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [34]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [35]:
frmae=1

In [36]:
import pickle
file1=open("data/pickle_dump.pkl","wb")
pickle.dump(frame,file=file1)
file1.close()
file1=open("data/pickle_dump.pkl","rb")
data1=pickle.load(file=file1)
file1.close()
!del data\pickle_dump.pkl  
data1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 文件必须关掉后才可以删除

## 数据库使用

+ 使用文本文件进行存储和读取的效率比较低，当数量非常大时，需要考虑使用数据库。
+ 下面介绍使用SQLite数据库，其通过python的sqlite3驱动
+ python和数据库之间的交互非常简单，特别是pandas提供了一些函数，使我们能轻松进行数据库操作

命令|功能
------|------
import sqlite3 |导入数据库包
conn = sqlite3.connect('data/mydatabase.db')| 创建在硬盘上面
conn = sqlite3.connect('"memory:') | 创建在内存上面

+ 其中conn对象是数据库链接对象，而对于数据库链接对象来说，具有以下操作：

命令|功能
------|------
        commit()     |       事务提交
        rollback()    |     事务回滚
        close()       |     关闭一个数据库链接
        cursor()      |    创建一个游标
      cu = conn.cursor() |   这样我们就创建了一个游标对象：cu. 在sqlite3中，所有sql语句的执行都要在游标对象的参与下完成      
    对于游标对象cu，具有以下具体操:|
        execute()        |执行一条sql语句
        executemany()   |执行多条sql语句
        close()         |游标关闭
        fetchone()     |从结果中取出一条记录
        fetchmany()    |从结果中取出多条记录
        fetchall()   |从结果中取出所有记录
        scroll()   |游标滚动    
   

+ 下面的例子在内存中创建了一个数据库，并生成数据库的一个表

In [37]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
#con = sqlite3.connect("test.db")
con.execute(query)
con.commit()

+ 将数据插入相应的表,使用sqlite中的函数

In [38]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

+ 我们可以使用pandas中的函数向数据库中插入一个表格

         import pandas.io.sql as sql

+ 在python2.7中，我们插入有中文关键字的DataFrame时需要
```
reload(sys)
sys.setdefaultencoding( "utf-8" )
```
+ to_sql函数可以插入一个DataFrame表格，注意如果表格存在，需要设置参数if_exists
    -  if_exists='fail' 如果存在就放弃插入
    -  if_exists='replace' 删掉原来的，创建新的表 
    -  if_exists='append' 插入数据

In [39]:
import pandas.io.sql as sql
#reload(sys)
#sys.setdefaultencoding( "utf-8" )
dataf=pd.read_csv("data/highf.csv",parse_dates=True,encoding="gbk")
datad=pd.read_csv("data/sh000001.csv",parse_dates=True,encoding="gbk")
sql.to_sql(dataf,"highfreq",con,if_exists="append")
sql.to_sql(datad,"daily",con,if_exists="append") 

C:\Users\heyb\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  


+ 从数据库中读取数据，用到了pandas中的函数

In [40]:
mysql="SELECT * from highfreq LIMIT 5"
sql.read_sql(mysql,con)

,index,Unnamed: 0,date,price,change,vol,amount,bid
0,0,0,15:00:05,4.39,NaN,3,1685,买盘
1,1,1,15:00:00,4.39,NaN,1651,724876,买盘
2,2,2,14:59:55,4.39,0.01,5987,2628661,买盘
3,3,3,14:59:45,4.38,-0.01,1242,544381,卖盘
4,4,4,14:59:40,4.39,NaN,3155,1385132,买盘


+ index_col 参数指定index

In [41]:

sql.read_sql('select * from daily limit 5', con,index_col=["date"])

,index,open,high,close,low,vol,amount
date,,,,,,,
1990-12-19,0,96.05,99.98,99.98,95.79,126000.0,494000.0
1990-12-20,1,104.30,104.39,104.39,99.98,19700.0,84000.0
1990-12-21,2,109.07,109.13,109.13,103.73,2800.0,16000.0
1990-12-24,3,113.57,114.55,114.55,109.13,3200.0,31000.0
1990-12-25,4,120.09,120.25,120.25,114.55,1500.0,6000.0


In [42]:
# 从highfreq
mysql="SELECT date, price from highfreq where bid='中性盘'  limit 5"
sql.read_sql(mysql,con)

,date,price
0,14:53:35,4.35
1,13:24:20,4.51
2,10:48:02,4.42
3,09:33:17,4.47
4,09:32:42,4.48


In [43]:
# 选择价格变化大于幅度0.03，成交在50手以上的交易
mysql="SELECT * from highfreq where abs(change)>0.03 and vol>50  limit 5"
sql.read_sql(mysql,con)

,index,Unnamed: 0,date,price,change,vol,amount,bid
0,2512,2512,09:32:27,4.49,0.04,838,376262,买盘
1,2523,2523,09:31:37,4.44,-0.05,21287,9451472,卖盘
2,2542,2542,09:25:07,4.46,4.46,25902,11552439,买盘


课后材料：了解下列网站中介绍的金融python包。

https://financetrain.com/best-python-librariespackages-finance-financial-data-scientists/